In [56]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import json
from typing import Dict, List, Tuple
from tqdm import tqdm
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.ndimage import gaussian_filter1d

# Set style for all plots
plt.style.use('default')
plt.rcParams.update({
    'figure.figsize': [8.0, 6.0],
    'figure.dpi': 300,
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white'
})


In [2]:
 # Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Setup paths
scratch_dir = os.getenv("SCRATCH_DIR")

In [3]:
# %%
# Base directory for analysis outputs
OUTPUT_DIR = Path(scratch_dir) / "output"
SAVE_DIR = Path(OUTPUT_DIR) / "final_figures"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# Groups and their data paths
GROUPS = ['affair', 'paranoia', 'combined']

# Colors for plotting
COLORS = {
    'affair': '#e41a1c',      # Red
    'affair_light': '#ff6666', # Light red
    'paranoia': '#377eb8',    # Blue
    'paranoia_light': '#6699ff', # Light blue
    'combined': '#4daf4a',    # Green
    'combined_light': '#90ee90' # Light green
}

In [4]:
def save_figure(fig, save_dir, filename, dpi=300, bbox_inches='tight'):
    # Remove any existing file extension from the filename
    base_filename = Path(filename).stem
    
    # Save both PNG and SVG versions
    fig.savefig(Path(save_dir) / f'{base_filename}.png', dpi=dpi, bbox_inches=bbox_inches)
    fig.savefig(Path(save_dir) / f'{base_filename}.svg', dpi=dpi, bbox_inches=bbox_inches)
    plt.close(fig)

In [5]:
SAVE_DIR_FIG2 = Path(SAVE_DIR) / "figure2"
SAVE_DIR_FIG2.mkdir(parents=True, exist_ok=True)

In [14]:
def load_model(group: str, n_state: int) -> Dict:
    """Load both metrics and summary files."""
    base_path = OUTPUT_DIR / f"{group}_hmm_{n_state}states_ntw_native_trimmed" / "models"
    
    # Load metrics
    model_path = base_path / f"{group}_hmm_model.pkl"
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

In [15]:
def load_data(group: str, n_state: int) -> Dict:
    """Load both metrics and summary files."""
    base_path = OUTPUT_DIR / f"{group}_hmm_{n_state}states_ntw_native_trimmed" / "statistics"
    
    # Load metrics
    metrics_path = base_path / f"{group}_metrics.pkl"
    try:
        with open(metrics_path, 'rb') as f:
            metrics = pickle.load(f)
    except Exception as e:
        print(f"Error loading metrics for {group}, n_state={n_state}: {e}")
        metrics = None
    
    # Load summary
    summary_path = base_path / f"{group}_summary.json"
    try:
        with open(summary_path, 'r') as f:
            summary = json.load(f)
    except Exception as e:
        print(f"Error loading summary for {group}, n_state={n_state}: {e}")
        summary = None
    
    return {'metrics': metrics, 'summary': summary}

In [36]:
combined_result = load_data(group="combined", n_state=3)
combined_model = load_model(group="combined", n_state=3)
affair_result = load_data(group="affair", n_state=3)
paranoia_result = load_data(group="paranoia", n_state=3)


In [41]:
affair_result['metrics'].keys()

dict_keys(['state_metrics', 'state_properties', 'subject_consistency'])

In [42]:
affair_result['metrics']['state_metrics'].keys()

dict_keys(['subject_level', 'group_level', 'temporal', 'uncertainty'])

In [47]:
affair_result['metrics']['state_metrics']['subject_level'][0].keys()

dict_keys(['durations', 'transitions', 'frequencies', 'fractional_occupancy', 'duration_stats'])

In [53]:
affair_result['metrics']['state_metrics']['temporal'].keys()

dict_keys(['switching_rate', 'state_mixing', 'recurrence_intervals', 'state_stability', 'entropy_rate', 'summary', 'raw'])

In [49]:
affair_result['metrics']['state_metrics']['uncertainty'].keys()

dict_keys(['state_occupancy', 'transition_rates', 'temporal_metrics'])

In [51]:
affair_result['metrics']['state_metrics']['uncertainty']['state_occupancy'][0].keys()

dict_keys(['ci_lower', 'ci_upper', 'std'])

In [44]:
affair_result['metrics']['state_properties'].keys()

dict_keys([0, 1, 2, 'separability'])

In [45]:
affair_result['metrics']['state_properties'][0].keys()

dict_keys(['mean_pattern', 'mean_pattern_ci', 'std_pattern', 'cv_pattern', 'covariance', 'correlation', 'effective_dimension', 'feature_importance', 'top_features', 'pattern_stability'])

In [52]:
affair_result['metrics']['state_properties']['separability'].keys()

dict_keys(['pairwise_distances', 'confusion_matrix', 'mahalanobis_distances', 'bootstrap_metrics', 'summary'])

In [38]:
paranoia_result

{'metrics': {'state_metrics': {'subject_level': {0: {'durations': {0: [4.5,
       4.5,
       10.5,
       16.5,
       13.5,
       9.0,
       12.0,
       16.5,
       12.0,
       4.5,
       6.0,
       10.5,
       6.0,
       16.5,
       6.0,
       16.5,
       3.0,
       21.0,
       16.5,
       13.5,
       4.5,
       4.5,
       12.0,
       12.0,
       4.5,
       16.5],
      1: [45.0,
       4.5,
       10.5,
       10.5,
       3.0,
       3.0,
       13.5,
       12.0,
       12.0,
       4.5,
       4.5,
       4.5,
       7.5,
       7.5,
       4.5,
       7.5,
       1.5],
      2: [7.5,
       6.0,
       21.0,
       7.5,
       6.0,
       4.5,
       24.0,
       6.0,
       6.0,
       6.0,
       19.5,
       1.5,
       4.5,
       3.0,
       21.0,
       3.0,
       3.0,
       7.5,
       6.0,
       10.5,
       3.0,
       7.5,
       7.5,
       9.0,
       4.5,
       4.5,
       12.0,
       16.5,
       9.0]},
     'transitions': array([[0.    

In [54]:
state_mapping = {
    'affair_to_paranoia': {0: 1, 1: 2, 2: 0},  # affair state 0 maps to paranoia state 1, etc.
    'paranoia_to_affair': {1: 0, 2: 1, 0: 2}   # paranoia state 1 maps to affair state 0, etc.
}

In [57]:
def compare_state_temporal_dynamics(affair_data, paranoia_data, state_mapping):
    """Compare temporal dynamics of aligned states between groups."""
    results = {}
    
    # Duration statistics comparison
    for affair_state, paranoia_state in state_mapping['affair_to_paranoia'].items():
        # Collect durations for each subject in each group
        affair_durations = [subj['duration_stats'][affair_state]['mean'] 
                           for subj in affair_data['subject_level'].values()]
        paranoia_durations = [subj['duration_stats'][paranoia_state]['mean'] 
                             for subj in paranoia_data['subject_level'].values()]
        
        # Statistical test
        t_stat, p_value = stats.ttest_ind(affair_durations, paranoia_durations)
        
        # Calculate effect size (Cohen's d)
        effect_size = (np.mean(paranoia_durations) - np.mean(affair_durations)) / \
                     np.sqrt((np.std(paranoia_durations)**2 + np.std(affair_durations)**2) / 2)
        
        results[f'state_{affair_state}_vs_{paranoia_state}'] = {
            'affair_mean_duration': np.mean(affair_durations),
            'paranoia_mean_duration': np.mean(paranoia_durations),
            'p_value': p_value,
            'effect_size': effect_size
        }
    
    return results

In [62]:
temporal_results = compare_state_temporal_dynamics(affair_result['metrics']['state_metrics'], paranoia_result['metrics']['state_metrics'], state_mapping)
temporal_results

{'state_0_vs_1': {'affair_mean_duration': 11.937386573477127,
  'paranoia_mean_duration': 11.306402650375679,
  'p_value': 0.7083877942290828,
  'effect_size': -0.12566624034807988},
 'state_1_vs_2': {'affair_mean_duration': 9.069103707497838,
  'paranoia_mean_duration': 8.187279324150428,
  'p_value': 0.06454675033011979,
  'effect_size': -0.6356130020496726},
 'state_2_vs_0': {'affair_mean_duration': 9.406109066102157,
  'paranoia_mean_duration': 10.163984441900576,
  'p_value': 0.31602083042620616,
  'effect_size': 0.3389425216441241}}

In [63]:
def compare_state_patterns(affair_states, paranoia_states, state_mapping):
    """Compare neural patterns of aligned states between groups."""
    results = {}
    
    for affair_state, paranoia_state in state_mapping['affair_to_paranoia'].items():
        # Get state patterns
        affair_pattern = affair_states[affair_state]['mean_pattern']
        paranoia_pattern = paranoia_states[paranoia_state]['mean_pattern']
        
        # Calculate similarity
        pattern_corr = np.corrcoef(affair_pattern, paranoia_pattern)[0, 1]
        
        # Compare feature importance
        affair_features = affair_states[affair_state]['top_features']['indices']
        paranoia_features = paranoia_states[paranoia_state]['top_features']['indices']
        shared_features = set(affair_features).intersection(set(paranoia_features))
        
        results[f'state_{affair_state}_vs_{paranoia_state}'] = {
            'pattern_correlation': pattern_corr,
            'affair_dimension': affair_states[affair_state]['effective_dimension'],
            'paranoia_dimension': paranoia_states[paranoia_state]['effective_dimension'],
            'shared_important_features': len(shared_features),
            'affair_stability': affair_states[affair_state]['pattern_stability'],
            'paranoia_stability': paranoia_states[paranoia_state]['pattern_stability']
        }
    
    return results

In [64]:
pattern_results = compare_state_patterns(affair_result['metrics']['state_properties'], paranoia_result['metrics']['state_properties'], state_mapping)
pattern_results

{'state_0_vs_1': {'pattern_correlation': 0.9451349704767232,
  'affair_dimension': 3.5097216681838495,
  'paranoia_dimension': 3.634608579670044,
  'shared_important_features': 10,
  'affair_stability': 0.9781190529234535,
  'paranoia_stability': 0.9897357407406784},
 'state_1_vs_2': {'pattern_correlation': 0.46798008803648633,
  'affair_dimension': 5.104726977346941,
  'paranoia_dimension': 5.029674821163042,
  'shared_important_features': 7,
  'affair_stability': 0.9726470864932794,
  'paranoia_stability': 0.9766442608990831},
 'state_2_vs_0': {'pattern_correlation': 0.33331942206505116,
  'affair_dimension': 4.595835500805606,
  'paranoia_dimension': 6.282833602307736,
  'shared_important_features': 8,
  'affair_stability': 0.9689666069538728,
  'paranoia_stability': 0.9185088407979235}}

In [65]:
def compare_transition_dynamics(affair_data, paranoia_data, state_mapping):
    """Compare state transition patterns between groups."""
    results = {}
    
    # Calculate average transition matrices for each group
    affair_transitions = np.mean([subj['transitions'] for subj in affair_data['subject_level'].values()], axis=0)
    paranoia_transitions = np.mean([subj['transitions'] for subj in paranoia_data['subject_level'].values()], axis=0)
    
    # Align paranoia transitions to affair state order
    paranoia_aligned = np.zeros_like(paranoia_transitions)
    for i in range(3):
        for j in range(3):
            paranoia_i = state_mapping['affair_to_paranoia'][i]
            paranoia_j = state_mapping['affair_to_paranoia'][j]
            paranoia_aligned[i, j] = paranoia_transitions[paranoia_i, paranoia_j]
    
    # Calculate transition differences
    transition_diff = paranoia_aligned - affair_transitions
    
    # Calculate transition entropy for each state
    affair_entropy = [-np.sum(affair_transitions[i] * np.log(affair_transitions[i] + 1e-10)) 
                     for i in range(3)]
    paranoia_entropy = [-np.sum(paranoia_aligned[i] * np.log(paranoia_aligned[i] + 1e-10)) 
                       for i in range(3)]
    
    results = {
        'affair_transitions': affair_transitions,
        'paranoia_aligned': paranoia_aligned,
        'transition_diff': transition_diff,
        'affair_entropy': affair_entropy,
        'paranoia_entropy': paranoia_entropy
    }
    
    return results

In [66]:
transition_results = compare_transition_dynamics(affair_result['metrics']['state_metrics'], paranoia_result['metrics']['state_metrics'], state_mapping)
transition_results
    


{'affair_transitions': array([[0.        , 0.59312083, 0.40687917],
        [0.56319933, 0.        , 0.43680067],
        [0.69559695, 0.30440305, 0.        ]]),
 'paranoia_aligned': array([[0.        , 0.54521115, 0.45478885],
        [0.61731935, 0.        , 0.38268065],
        [0.56960794, 0.43039206, 0.        ]]),
 'transition_diff': array([[ 0.        , -0.04790968,  0.04790968],
        [ 0.05412003,  0.        , -0.05412003],
        [-0.12598902,  0.12598902,  0.        ]]),
 'affair_entropy': [0.6757025247284174, 0.6851374621811666, 0.614548963601129],
 'paranoia_entropy': [0.6890534950467997,
  0.6653611956536445,
  0.6834251032629806]}

In [67]:
def compare_subject_consistency(affair_data, paranoia_data, state_mapping):
    """Compare how consistently states appear across subjects in each group."""
    # Extract fractional occupancy for each subject in each group
    affair_occupancy = np.array([list(subj['fractional_occupancy']) 
                               for subj in affair_data['subject_level'].values()])
    paranoia_occupancy = np.array([list(subj['fractional_occupancy']) 
                                 for subj in paranoia_data['subject_level'].values()])
    
    # Align paranoia occupancy to affair state order
    paranoia_aligned = np.zeros_like(paranoia_occupancy)
    for i in range(3):
        paranoia_i = state_mapping['affair_to_paranoia'][i]
        paranoia_aligned[:, i] = paranoia_occupancy[:, paranoia_i]
    
    # Calculate variance across subjects for each state
    affair_variance = np.var(affair_occupancy, axis=0)
    paranoia_variance = np.var(paranoia_aligned, axis=0)
    
    # Calculate average pairwise correlation between subjects
    affair_correlations = []
    paranoia_correlations = []
    
    for i in range(len(affair_occupancy)):
        for j in range(i+1, len(affair_occupancy)):
            affair_correlations.append(np.corrcoef(affair_occupancy[i], affair_occupancy[j])[0, 1])
    
    for i in range(len(paranoia_aligned)):
        for j in range(i+1, len(paranoia_aligned)):
            paranoia_correlations.append(np.corrcoef(paranoia_aligned[i], paranoia_aligned[j])[0, 1])
    
    return {
        'affair_state_variance': affair_variance,
        'paranoia_state_variance': paranoia_variance,
        'affair_subject_correlation': np.mean(affair_correlations),
        'paranoia_subject_correlation': np.mean(paranoia_correlations)
    }

In [69]:
consistency_results = compare_subject_consistency(affair_result['metrics']['state_metrics'], paranoia_result['metrics']['state_metrics'], state_mapping)
consistency_results

{'affair_state_variance': array([0.0236078 , 0.00456026, 0.0123093 ]),
 'paranoia_state_variance': array([0.02859393, 0.00479595, 0.01243271]),
 'affair_subject_correlation': 0.1872488137562338,
 'paranoia_subject_correlation': 0.10842534819789874}